In [1]:
import os
import os.path as op
import subprocess

import numpy as np
import pandas as pd

In [ ]:
WORKING_PATH = "..."  # Set your working path here.

beh = pd.read_csv(
    op.join(WORKING_PATH, "data/beh/beh_residualized_extended_050225.csv")
)
aparc = pd.read_csv(
    op.join(WORKING_PATH, "data/fs-measures/aparcstats2table_combined.csv")
)
aseg = pd.read_csv(
    op.join(WORKING_PATH, "data/fs-measures/aseg_stats_combined.csv")
).rename(columns={"Measure:volume": "id"})

beh = beh[beh["site"] == "regensburg"]
db = pd.merge(beh, aparc, on="id", how="inner").merge(aseg, on="id", how="inner")
db["sex_cycle"] = db["sex"] + db["cycle"].str.capitalize()
db["sex_cycle"] = db["sex_cycle"].replace({"maleMale": "male"})
db

In [3]:
FSGD_PATH = op.join(WORKING_PATH, "whole-brain_regensburg_cycle/data")
CORTISOL_TYPES = ["increase"]


def add_header_to_tsv(df, file_path, header_template, header_addition):
    THINGS_TO_REMOVE = {
        "Unnamed: 0": "Variables",
        "Unnamed: 1": "",
        "Unnamed: 2": "",
    }

    # Convert the DataFrame to a TSV string
    tsv_content = df.to_csv(sep="\t", index=False)

    # Remove unwanted elements
    for item in THINGS_TO_REMOVE:
        tsv_content = tsv_content.replace(item, THINGS_TO_REMOVE[item])

    # Prepend the header template to the TSV content
    modified_content = header_template + header_addition + tsv_content

    # Write the modified content back to the file
    with open(file_path, "w") as file:
        file.write(modified_content)

In [4]:
HEADER_TEMPLATE = """GroupDescriptorFile 1
Title Cortisol
Class male
Class femaleLuteal
Class femalePill
Class femaleMenopause
DeMeanFlag 1
ReScaleFlag 1
"""

fsgd_path = op.join(FSGD_PATH, "fsgd")
os.makedirs(fsgd_path, exist_ok=True)

CYCLE_DUMMY = {
    "male": [0, 0, 0],
    "luteal": [1, 0, 0],
    "pill": [1, 1, 0],
    "menopause": [1, 1, 1],
}

cycle = np.array([CYCLE_DUMMY[cycle] for cycle in db["cycle"]])

for cortisol_type in CORTISOL_TYPES:
    fsgd = pd.concat(
        [
            db[["id", "sex_cycle", cortisol_type, "age"]],
        ],
        axis=1,
    )  # 2 Groups FSGD
    fsgd.insert(0, "", "Input")
    fsgd.columns = [""] * len(fsgd.columns)
    header_addition = f"Variables {cortisol_type} age"
    fsgd_file = op.join(fsgd_path, f"fsgd_{cortisol_type}.tsv")
    add_header_to_tsv(
        fsgd,
        fsgd_file,
        HEADER_TEMPLATE,
        header_addition,
    )
    subprocess.run(
        "tr '\r' '\n' < {} > {}".format(fsgd_file, fsgd_file.replace(".tsv", ".fsgd")),
        shell=True,
    )

# Contrast
contrasts = {
    "average": [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
    "men>women": [0, 0, 0, 0, 1, -1, -1, -1, 0, 0, 0, 0],
    "women>men": [0, 0, 0, 0, -1, 1, 1, 1, 0, 0, 0, 0],
    "men": [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    "women": [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
}
for cont in contrasts:
    out_file = op.join(fsgd_path, f"contrast_{cont}.mtx")
    np.savetxt(
        out_file,
        np.array([contrasts[cont]]),  # Save as a row vector
        fmt="%d",
    )